# `wandb` sweep definition creator

## Imports

In [1]:
import wandb

import yaml

from dotmap import DotMap

## Helper functions

In [2]:
def add_parameters_key(dictionary):
    # Add a key parameters to every nested dictionary in a dictionary
    for key, value in dictionary.items():
        if isinstance(value, dict):
            if 'values' not in value.keys() and 'min' not in value.keys() and 'max' not in value.keys():
                dictionary[key] = {'parameters': value}
                add_parameters_key(value)
            
def add_value_key(dictionary):
    # Add a key value to every nested value in a dictionary
    for key, value in dictionary.items():
        if isinstance(value, dict):
            add_value_key(value)
        else:
            dictionary[key] = {'value': value}

def remove_value_key_if_more_keys_exist(dictionary):
    # Remove a key value to every nested value in a dictionary if there are more keys
    for key, value in dictionary.items():
        if isinstance(value, dict):
            if 'value' in value.keys() and len(value.keys()) > 1:
                del value['value']
            remove_value_key_if_more_keys_exist(value)

def merge_dicts(dict1: dict, dict2: dict):
    """Code taken from: https://stackoverflow.com/a/58742155/11251769."""

    for key, val in dict1.items():
        if type(val) == dict:
            if key in dict2 and type(dict2[key] == dict):
                merge_dicts(dict1[key], dict2[key])
        else:
            if key in dict2:
                dict1[key] = dict2[key]

    for key, val in dict2.items():
        if not key in dict1:
            dict1[key] = val

    return dict1

## Base config

In [3]:
cfg = DotMap()

cfg.criterion.name = 'CrossEntropyLoss'

cfg.meta_learning.meta_batch_size = 32
cfg.meta_learning.name = 'supervised-cosine'
cfg.meta_learning.ways = 5
cfg.meta_learning.shots = 1
cfg.meta_learning.query_shots = 1

cfg.model.name = 'TCN'
cfg.model.cfg = DotMap({'input_size': 1, 'channel_sizes': [25]*8, 'kernel_size': 7, 'dropout': 0.1, 'batchnorm': False})

cfg.optimizer.name = 'SGD'
cfg.optimizer.cfg = DotMap({"lr": 0.3, "momentum": 0.9, "weight_decay": 5e-4, })

cfg.seed = 4223747124

cfg.task.name = 'Omniglot'
cfg.task.cfg = DotMap({'layout': 'pixel', 'random_rotation': [0, 90, 180, 270], 'transform_test': False, 'rotation_extended_dataset': True}) #  
cfg.task.train = DotMap({'n_epochs': 500, 'batch_size': 964})
cfg.task.val = DotMap({'percentage':0.1})

## Variable config

In [4]:
v_cfg = DotMap()

v_cfg.model.cfg = DotMap({ 'dropout': {'max': 0.6, 'min': 0.0},})

v_cfg.optimizer.name = {'values': ['SGD', 'RMSprop']}
v_cfg.optimizer.cfg = DotMap({"lr": {'max': -1.897, 'min': -9.210, 'distribution': 'log_uniform'}, "momentum": {'max': 1.0, 'min': 0.0}, "weight_decay": {'max': -2.3025, 'min': -13.815, 'distribution': 'log_uniform'} })

v_cfg.task.train = DotMap({'batch_size': {'values': [64, 256, 512, 1024, 2048]}})


## Sweep config

In [5]:
sweep_cfg = DotMap()

sweep_cfg.method = 'bayes'
sweep_cfg.name = 'omniglot_tcn'
sweep_cfg.program = 'runner.py'
sweep_cfg.metric = {'name': "meta_val/accuracy", 'goal': 'maximize'}
sweep_cfg.early_terminate = {'type': 'hyperband', 'min_iter': 100, 's': 2}

## Merging and fixing dictionaries

In [6]:
sweep_parameters_cfg = cfg.toDict()

add_parameters_key(sweep_parameters_cfg)
add_value_key(sweep_parameters_cfg)

In [7]:
sweep_variable_parameters_cfg = v_cfg.toDict()

add_parameters_key(sweep_variable_parameters_cfg)

In [8]:
full_sweep_parameters_cfg = merge_dicts(sweep_parameters_cfg, sweep_variable_parameters_cfg)
remove_value_key_if_more_keys_exist(full_sweep_parameters_cfg)

sweep_cfg.parameters = full_sweep_parameters_cfg

## Show final config

In [9]:
print(yaml.dump(sweep_cfg.toDict(), default_flow_style=False))

early_terminate:
  min_iter: 100
  s: 2
  type: hyperband
method: bayes
metric:
  goal: maximize
  name: meta_val/accuracy
name: omniglot_tcn
parameters:
  criterion:
    parameters:
      name:
        value: CrossEntropyLoss
  meta_learning:
    parameters:
      meta_batch_size:
        value: 32
      name:
        value: supervised-cosine
      query_shots:
        value: 1
      shots:
        value: 1
      ways:
        value: 5
  model:
    parameters:
      cfg:
        parameters:
          batchnorm:
            value: false
          channel_sizes:
            value:
            - 25
            - 25
            - 25
            - 25
            - 25
            - 25
            - 25
            - 25
          dropout:
            max: 0.6
            min: 0.0
          input_size:
            value: 1
          kernel_size:
            value: 7
      name:
        value: TCN
  optimizer:
    parameters:
      cfg:
        parameters:
          lr:
            distribution

## Submit config

In [10]:
sweep_id = wandb.sweep(project="meta-learning-arena", entity="douwe", sweep=sweep_cfg.toDict())

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: h53zqd0i
Sweep URL: https://wandb.ai/douwe/meta-learning-arena/sweeps/h53zqd0i
